# Audio collection

We have a set of texts, and we need to get voice recordings of these texts. We ask performers to read the texts aloud and record themselves. Recordings like these are used for training voice assistants

To get acquainted with Toloka tools for free, you can use the promo code **TOLOKAKIT1** on $20 on your [profile page](https://toloka.yandex.com/requester/profile?utm_source=github&utm_medium=site&utm_campaign=tolokakit) after registration.

### Call to action
If you found some bugs or have a new feature idea, don't hesitate to [open a new issue on Github](https://github.com/Toloka/toloka-kit/issues/new/choose).
Like our library and examples? Star [our repo on Github](https://github.com/Toloka/toloka-kit)

Prepare the environment and import everything you'll need.

In [ ]:
%%capture
!pip install toloka-kit==0.1.26
!pip install ipython

import datetime
import logging
import sys
import time
import getpass

import toloka.client as toloka
import toloka.client.project.template_builder as tb

import IPython.display as display
import pandas

In [ ]:
logging.basicConfig(
    format='[%(levelname)s] %(name)s: %(message)s',
    level=logging.INFO,
    stream=sys.stdout,
)

Сreate toloka-client instance. All api calls will go through it. More about OAuth token in our [Learn the basics example](https://github.com/Toloka/toloka-kit/tree/main/examples/0.getting_started/0.learn_the_basics) [![Open In Colab](https://colab.research.google.com/assets/colab-badge.svg)](https://colab.research.google.com/github/Toloka/toloka-kit/blob/main/examples/0.getting_started/0.learn_the_basics/learn_the_basics.ipynb)

In [ ]:
toloka_client = toloka.TolokaClient(getpass.getpass('Enter your OAuth token: '), 'PRODUCTION') # Or switch to 'SANDBOX'
print(toloka_client.get_requester())

---
## Create a project

> Note: The project name and description will be visible to the performers

In [ ]:
new_project = toloka.Project(
    public_name='Voice recording',
    public_description='Tap the voice recorder button and read the text aloud.',
)

Prepare the task interface.

The task interface should:
- Contain the text, that performer should read.
- Permit recording audio.

Read more about the [Template Builder](https://toloka.ai/docs/template-builder/index.html?utm_source=github&utm_medium=site&utm_campaign=tolokakit) in the Requester’s Guide.

Check the [Interface section](https://toloka.ai/knowledgebase/interface?utm_source=github&utm_medium=site&utm_campaign=tolokakit) of our Knowledge Base for more tips on interface design

In [ ]:
text_view = tb.TextViewV1(tb.InputData('text'))
audio_field = tb.AudioFieldV1(tb.OutputData('audio_file'), validation=tb.RequiredConditionV1())
width_plugin = tb.TolokaPluginV1('scroll', task_width=500)

project_interface = toloka.project.TemplateBuilderViewSpec(
    view=tb.ListViewV1(items=[text_view, audio_field]),
    plugins=[width_plugin]
)

Set data specification. And set task interface to project.

> Note: Specifications are a description of input data that will be used in a project and the output data that will be collected from the performers

Read more about [input and output data specifications](https://yandex.ru/support/toloka-tb/operations/create-specs.html?utm_source=github&utm_medium=site&utm_campaign=tolokakit) in the Requester’s Guide.

In [ ]:
input_specification = {'text': toloka.project.StringSpec()}
output_specification = {'audio_file': toloka.project.FileSpec()}

new_project.task_spec = toloka.project.task_spec.TaskSpec(
        input_spec=input_specification,
        output_spec=output_specification,
        view_spec=project_interface,
)

Write comprehensive instructions. Be sure to describe your task to performers who record words and phrases via different devices (iOS and Android). Mention how long it will take to check completed tasks (max time)

Get more tips on designing [instructions](https://toloka.ai/knowledgebase/instruction?utm_source=github&utm_medium=site&utm_campaign=tolokakit) in our Knowledge Base.

In [ ]:
new_project.public_instructions = """Each task contains words and phrases. You need to read and record them.
Make a new recording for each phrase, even if it has already been used in other tasks."""

Create a project via API request.

In [ ]:
new_project = toloka_client.create_project(new_project)

## Create the main pool
A pool is a set of paid tasks grouped into task pages. These tasks are sent out for completion at the same time.

> Note: All tasks within a pool have the same settings (price, quality control, etc.)

Let's set pool parameters:
   - Give the pool any name you find suitable. You are the only one who will see it. The description can be either public or private.
   - The price per task suite to $0.01. Audio recording tasks are normally paid as basic tasks (e.g. binary classification) because these tasks do not take much time. Read more about [pricing principles](https://toloka.ai/knowledgebase/pricing?utm_source=github&utm_medium=site&utm_campaign=tolokakit) in our Knowledge Base.
   - Allows only English-speaking performers do these tasks. And allow performers to complete tasks on their computers or mobile devices.
   - Auto accept solutions to False. Since there is no one true answer to a voice recording task that can be used as ground truth, post-acceptance is the preferable way to check if the recordings provided are acceptable. Read more about [quality control principles](https://toloka.ai/knowledgebase/quality-control?utm_source=github&utm_medium=site&utm_campaign=tolokakit) in our Knowledge Base or check out [post-acceptance settings](https://toloka.ai/docs/guide/concepts/offline-accept.html?utm_source=github&utm_medium=site&utm_campaign=tolokakit) in the Requester’s Guide
   - Time given to complete a task suite to 120 seconds. To understand how much time it should take to complete a task suite, try doing it yourself.
   - Overlap to 1. In the case of voice recording tasks, you only need one correct response. This overlap value is set by default, so we don't need to define it in code.

In [ ]:
new_pool = toloka.Pool(
    project_id=new_project.id,
    private_name='Voice recording',
    may_contain_adult_content=False,
    will_expire=datetime.datetime.utcnow() + datetime.timedelta(days=365),
    reward_per_assignment=0.01,
    assignment_max_duration_seconds=60*10,
    auto_accept_solutions=False,
    auto_accept_period_day=1,
    filter=(
        (toloka.filter.Languages.in_('EN')) &
        (toloka.filter.ClientType == 'TOLOKA_APP')
    ),
)

Let's specify the number of tasks per page. In the case of audio recording tasks several tasks per page are preferable, so that performers will complete tasks faster.

In [ ]:
new_pool.set_mixer_config(real_tasks_count=5)

Create a pool.

In [ ]:
new_pool = toloka_client.create_pool(new_pool)

## Add a tasks and run the project

We will use texts from [Noisy speech database](https://datashare.ed.ac.uk/handle/10283/2791) that distributed under a Creative Commons Attribution 4.0 International license
[![License: CC BY 4.0](https://img.shields.io/badge/License-CC%20BY%204.0-lightgrey.svg)](https://creativecommons.org/licenses/by/4.0/).

> **Citation:** Valentini-Botinhao, Cassia. (2017). Noisy speech database for training speech enhancement algorithms and TTS models, 2016 [sound]. University of Edinburgh. School of Informatics. Centre for Speech Technology Research (CSTR). https://doi.org/10.7488/ds/2117.

In [32]:
!curl https://tlk.s3.yandex.net/ext_dataset/noisy_speech/noisy_speech.tsv --output dataset.tsv

dataset = pandas.read_csv('dataset.tsv', sep='\t')

print(dataset['text'])

  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
100   98k  100   98k    0     0   323k      0 --:--:-- --:--:-- --:--:--  323k
0      He said that healthy eating was high on the co...
1                       It was deployed in the Gulf War.
2      They were under a lot of pressure from the oth...
3                         It's the real thing, for sure.
4                            You are not going in blind.
                             ...                        
819    He's not written as a terribly exciting person...
820             I think he's the favourite, no question.
821                                       I was furious.
822    The announcement was made after inquiries from...
823                       We've made a couple of albums.
Name: text, Length: 824, dtype: object


Let's create tasks and upload them to our pool. Then starting the pool.

> Note: Remember that the tasks will be completed by actual Tolokers. Double check that everything is correct with your project configuration before you start the pool

In [ ]:
tasks = [
    toloka.Task(input_values={'text': text}, pool_id=new_pool.id)
    for text in dataset['text'].values[:20]
]

toloka_client.create_tasks(tasks, allow_defaults=True)

new_pool = toloka_client.open_pool(new_pool.id)

You can open pool in web-interface and preview preformers interface.

<table  align="center">
  <tr><td>
    <img src="./img/tasks_preview.png"
         alt="Task page preview"  width="1000">
  </td></tr>
  <tr><td align="center">
    <b>Figure 1.</b> What the task page preview can looks like.
  </td></tr>
</table>

## Receiving responses

Wait for performers to complete the tasks, then download the results.

In [ ]:
def wait_pool_for_close(pool_id, minutes_to_wait=1):
    sleep_time = 60 * minutes_to_wait
    pool = toloka_client.get_pool(pool_id)
    while not pool.is_closed():
        op = toloka_client.get_analytics([toloka.analytics_request.CompletionPercentagePoolAnalytics(subject_id=pool.id)])
        op = toloka_client.wait_operation(op)
        percentage = op.details['value'][0]['result']['value']
        print(
            f'   {datetime.datetime.now().strftime("%H:%M:%S")}\t'
            f'Pool {pool.id} - {percentage}%'
        )
        time.sleep(sleep_time)
        pool = toloka_client.get_pool(pool.id)
    print('Pool was closed.')

wait_pool_for_close(new_pool.id)

Since the main quality control method for this kind of task is post-acceptance, you will need to review the tasks after the pool is completed.

You can check the quality of responses and reject and reevaluate incorrect assignments. Performers will get paid only after their assignment is accepted.

You can open the pool page and click Review Assignments, for manual checking responses from performers.

Another way to review tasks is to ask other performers to do that. We recommend this option when you have limited resources for checking tasks yourself. Examples of such checking projects you can find in our [Object detection example](https://github.com/Toloka/toloka-kit/tree/main/examples/1.computer_vision/object_detection) [![Open In Colab](https://colab.research.google.com/assets/colab-badge.svg)](https://colab.research.google.com/github/Toloka/toloka-kit/blob/main/examples/1.computer_vision/object_detection/object_detection.ipynb) or [Questing answering on SQuAD example](https://github.com/Toloka/toloka-kit/tree/main/examples/SQUAD2.0) [![Open In Colab](https://colab.research.google.com/assets/colab-badge.svg)](https://colab.research.google.com/github/Toloka/toloka-kit/blob/main/examples/SQUAD2.0/SQUAD2.0_processing.ipynb)

Download the results.

> Note: You should download files' ids, not the files themselves. The files will only be needed right before reviewing.

In [ ]:
results_list = []

for assignment in toloka_client.get_assignments(pool_id=new_pool.id, status='SUBMITTED'):
    for task, solution in zip(assignment.tasks, assignment.solutions):
        results_list.append({**solution.output_values, **task.input_values})
results_iter = iter(results_list)
print(len(results_list))

Run the cell below multiple times to see different responses.

In [ ]:
res = next(results_iter, None)
if res is not None:
    with open('tmp_audio_file.wav', 'w+b') as out_f:
        toloka_client.download_attachment(res['audio_file'], out_f)
        print(res['text'])
else:
    print('No more results')

display.Audio('tmp_audio_file.wav', autoplay=True)

## Summary

This project consists of the minimum number of settings that will allow you to collect audio dataset.